## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Date
0,0,Kirtipur,NP,27.67,85.28,73.40,94,40,5.82,scattered clouds,2020-08-30 15:39:58
1,1,Qaanaaq,GL,77.48,-69.36,33.22,78,0,6.08,clear sky,2020-08-30 15:39:58
2,2,Bethel,US,41.37,-73.41,71.01,60,75,10.29,broken clouds,2020-08-30 15:39:27
3,3,Huilong,CN,31.81,121.66,86.00,92,0,3.00,light rain,2020-08-30 15:39:58
4,4,Busselton,AU,-33.65,115.33,45.00,67,0,4.74,clear sky,2020-08-30 15:39:59


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Date
3,3,Huilong,CN,31.81,121.66,86.00,92,0,3.00,light rain,2020-08-30 15:39:58
8,8,Bubaque,GW,11.28,-15.83,80.80,80,85,14.47,overcast clouds,2020-08-30 15:35:43
14,14,San Patricio,MX,19.22,-104.70,78.80,100,90,1.21,overcast clouds,2020-08-30 15:40:01
17,17,Axim,GH,4.87,-2.24,78.26,73,85,10.83,overcast clouds,2020-08-30 15:40:02
19,19,Pasighat,IN,28.07,95.33,77.99,77,100,2.82,overcast clouds,2020-08-30 15:40:03
23,23,Ribeira Grande,PT,38.52,-28.70,80.60,83,40,11.41,scattered clouds,2020-08-30 15:40:04
26,26,Castro,BR,-24.79,-50.01,80.28,40,0,7.56,clear sky,2020-08-30 15:40:05
29,29,Oistins,BB,13.07,-59.53,87.80,74,40,16.11,scattered clouds,2020-08-30 15:40:06
34,34,Morant Bay,JM,17.88,-76.41,89.60,70,20,5.82,few clouds,2020-08-30 15:40:07
35,35,Salalah,OM,17.02,54.09,78.80,88,90,2.24,overcast clouds,2020-08-30 15:40:08


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Date
3,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False
14,False,False,False,False,False,False,False,False,False,False,False
17,False,False,False,False,False,False,False,False,False,False,False
19,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
689,False,False,False,False,False,False,False,False,False,False,False
691,False,False,False,False,False,False,False,False,False,False,False
695,False,False,False,False,False,False,False,False,False,False,False
703,False,False,False,False,False,False,False,False,False,False,False


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
3,Huilong,CN,86.00,light rain,31.81,121.66,
8,Bubaque,GW,80.80,overcast clouds,11.28,-15.83,
14,San Patricio,MX,78.80,overcast clouds,19.22,-104.70,
17,Axim,GH,78.26,overcast clouds,4.87,-2.24,
19,Pasighat,IN,77.99,overcast clouds,28.07,95.33,
23,Ribeira Grande,PT,80.60,scattered clouds,38.52,-28.70,
26,Castro,BR,80.28,clear sky,-24.79,-50.01,
29,Oistins,BB,87.80,scattered clouds,13.07,-59.53,
34,Morant Bay,JM,89.60,few clouds,17.88,-76.41,
35,Salalah,OM,78.80,overcast clouds,17.02,54.09,


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
3,Huilong,CN,86.00,light rain,31.81,121.66,Hilton Garden Inn Qidong
8,Bubaque,GW,80.80,overcast clouds,11.28,-15.83,Casa Dora
14,San Patricio,MX,78.80,overcast clouds,19.22,-104.70,Hotel Cabo Blanco
17,Axim,GH,78.26,overcast clouds,4.87,-2.24,AXIM BEACH RESORT AND SPA
19,Pasighat,IN,77.99,overcast clouds,28.07,95.33,Siang Guest House


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt>{Current Weather}
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))